In [2]:
import os
# print(os.listdir("C://Users//navin//Dropbox//Ryerson University//CIND 820//complaints"))

import pandas as pd
import numpy as np
import zipfile
from scipy.stats import randint
import seaborn as sns # used for plot interactive graph. 
import matplotlib.pyplot as plt
import seaborn as sns
from io import StringIO

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.feature_selection import chi2
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn import metrics
#import warnings

In [3]:
df = pd.read_csv('complaints.csv')
#The dataset can be found on 
#https://catalog.data.gov/dataset/consumer-complaint-database
#our dataset is download on Sept 17, 2020
df.shape

(1757764, 18)

In [4]:
dfNew = df[['Product', 'Consumer complaint narrative']].copy()

# Remove missing values (NaN)
dfNew = dfNew[pd.notnull(dfNew['Consumer complaint narrative'])]
dfNew = dfNew[pd.notnull(dfNew['Product'])]
# Renaming second column for a simpler name
dfNew.columns = ['Product', 'Consumer_complaint'] 

dfNew.shape

(596333, 2)

In [5]:
dfNew.replace({'Product': 
             {'Credit reporting, credit repair services, or other personal consumer reports': 
              'Credit reporting, repair, or other reports', 
              'Credit reporting': 'Credit reporting, repair, or other reports',
              'Credit card': 'Credit card or prepaid card',
              'Prepaid card': 'Credit card or prepaid card',
              'Payday loan': 'Payday loan, title loan, or personal loan',
              'Money transfers': 'Money transfer, virtual currency, or money service',
              'Virtual currency': 'Money transfer, virtual currency, or money service'}}, 
            inplace= True)

In [6]:
dfNew.head(5)

,Product,Consumer_complaint
0,"Credit reporting, repair, or other reports",Although I am checking for and addressing miss...
1,Debt collection,transworld systems inc. \nis trying to collect...
3,Debt collection,"Over the past 2 weeks, I have been receiving e..."
5,Debt collection,Quickclickloans.com hired an attorney XXXX XXX...
6,"Credit reporting, repair, or other reports","Previously, on XX/XX/XXXX, XX/XX/XXXX, and XX/..."


In [7]:
dfNewOriginal=dfNew.copy()
#dfNew=dfNew[:1000]


In [8]:
#Random Sampling
dfNew=dfNewOriginal.sample(n=10000, random_state=6)
dfNew.head(10)


,Product,Consumer_complaint
1333694,"Credit reporting, repair, or other reports",I have disputed several times with Equifax and...
1382991,Debt collection,On XXXX XXXX I received a letter from first na...
374241,"Payday loan, title loan, or personal loan",Company : USA Cash Services I got an installme...
1488000,Credit card or prepaid card,"On Tuesday, XX/XX/2016 I logged into my accoun..."
19669,"Payday loan, title loan, or personal loan",WENT ON XXXX TODAY AND GOT AD FOR CONSUMER LOA...
1321083,Student loan,I 'm making payments of {$240.00} for XXXX sub...
702610,"Credit reporting, repair, or other reports","Transunion "" verified '' collections with XXXX..."
1300540,Debt collection,A medical Debt owed to XXXX in the original am...
1354293,"Credit reporting, repair, or other reports",XXXX is still showing on your credit report st...
1663457,Credit card or prepaid card,I have an XXXX store credit card and have spen...


In [9]:
#dfNew.to_csv ('complaints_samples.csv', index = False, header=True)



In [10]:
#dfNew = pd.read_csv('complaints_samples.csv')

PreProcessing

Lowercase

Remove Punctuation

Remove Stop words

Romove Digits

Limmatization


Lowecase

In [11]:
dfNew['Consumer_complaint'] =dfNew['Consumer_complaint'].apply(lambda x: ' '.join([i.lower() for i in x.split()]))

In [12]:
dfNew['Consumer_complaint'].head(10)

1333694    i have disputed several times with equifax and...
1382991    on xxxx xxxx i received a letter from first na...
374241     company : usa cash services i got an installme...
1488000    on tuesday, xx/xx/2016 i logged into my accoun...
19669      went on xxxx today and got ad for consumer loa...
1321083    i 'm making payments of {$240.00} for xxxx sub...
702610     transunion " verified '' collections with xxxx...
1300540    a medical debt owed to xxxx in the original am...
1354293    xxxx is still showing on your credit report st...
1663457    i have an xxxx store credit card and have spen...
Name: Consumer_complaint, dtype: object

In [13]:
import string
dfNew['Consumer_complaint'] =dfNew['Consumer_complaint'].apply(lambda x:' '.join([char for char in x.split() if char not in string.punctuation]))


In [14]:
dfNew['Consumer_complaint'].head(5)

1333694    i have disputed several times with equifax and...
1382991    on xxxx xxxx i received a letter from first na...
374241     company usa cash services i got an installment...
1488000    on tuesday, xx/xx/2016 i logged into my accoun...
19669      went on xxxx today and got ad for consumer loa...
Name: Consumer_complaint, dtype: object

In [15]:
dfNew['Consumer_complaint'] =dfNew['Consumer_complaint'].str.replace(r'[^\w\s]'," ")
dfNew['Consumer_complaint'].head(5)

1333694    i have disputed several times with equifax and...
1382991    on xxxx xxxx i received a letter from first na...
374241     company usa cash services i got an installment...
1488000    on tuesday  xx xx 2016 i logged into my accoun...
19669      went on xxxx today and got ad for consumer loa...
Name: Consumer_complaint, dtype: object

In [16]:

dfNew['Consumer_complaint'] = dfNew['Consumer_complaint'].str.replace(r"xx+\s"," ")
dfNew['Consumer_complaint'].head(5)


1333694    i have disputed several times with equifax and...
1382991    on   i received a letter from first national c...
374241     company usa cash services i got an installment...
1488000    on tuesday    2016 i logged into my account to...
19669      went on  today and got ad for consumer loan fr...
Name: Consumer_complaint, dtype: object

In [17]:
dfNew['Consumer_complaint'] = dfNew['Consumer_complaint'].str.replace(r"xx+[a-z]+\s"," ")
dfNew['Consumer_complaint'].head(5)

1333694    i have disputed several times with equifax and...
1382991    on   i received a letter from first national c...
374241     company usa cash services i got an installment...
1488000    on tuesday    2016 i logged into my account to...
19669      went on  today and got ad for consumer loan fr...
Name: Consumer_complaint, dtype: object

In [18]:
dfNew['Consumer_complaint'] = dfNew['Consumer_complaint'].str.replace(r"[a-z]+xx+\s"," ")
dfNew['Consumer_complaint'].head(5)

1333694    i have disputed several times with equifax and...
1382991    on   i received a letter from first national c...
374241     company usa cash services i got an installment...
1488000    on tuesday    2016 i logged into my account to...
19669      went on  today and got ad for consumer loan fr...
Name: Consumer_complaint, dtype: object

In [19]:
dfNew['Consumer_complaint'] = dfNew['Consumer_complaint'].str.replace(r"xxxx"," ")
dfNew['Consumer_complaint'].head(5)

1333694    i have disputed several times with equifax and...
1382991    on   i received a letter from first national c...
374241     company usa cash services i got an installment...
1488000    on tuesday    2016 i logged into my account to...
19669      went on  today and got ad for consumer loan fr...
Name: Consumer_complaint, dtype: object

In [20]:
#pip install nltk
from nltk.corpus import stopwords
stopE = stopwords.words('english')
dfNew['Consumer_complaint'] =dfNew['Consumer_complaint'].apply(lambda x: ' '.join([char for char in x.split() if char not in stopE]))

In [21]:
dfNew['Consumer_complaint'].head(5)

1333694    disputed several times equifax actual investig...
1382991    received letter first national collection bure...
374241     company usa cash services got installment loan...
1488000    tuesday 2016 logged account review went activi...
19669      went today got ad consumer loan one main ad in...
Name: Consumer_complaint, dtype: object

In [22]:
dfNew['Consumer_complaint'] =dfNew['Consumer_complaint'].apply(lambda x: ' '.join([char for char in x.split() if not char.isdigit()]))

dfNew['Consumer_complaint'] =dfNew['Consumer_complaint'].apply(lambda x: ' '.join([char for char in x.split() if char.isalpha()]))

In [23]:
dfNew['Consumer_complaint'].head(5)

1333694    disputed several times equifax actual investig...
1382991    received letter first national collection bure...
374241     company usa cash services got installment loan...
1488000    tuesday logged account review went activity st...
19669      went today got ad consumer loan one main ad in...
Name: Consumer_complaint, dtype: object

In [25]:
# pip install pattern
# Run this part twice
import pattern

from pattern.en import lemma
dfNew['Consumer_complaint'] =dfNew['Consumer_complaint'].apply(lambda x:' '.join([lemma(wd) for wd in x.split()]))

In [26]:
dfNew['Consumer_complaint'].head(5)

1333694    dispute several time equifax actual investigat...
1382991    receive letter first national collection burea...
374241     company usa cash service get installment loan ...
1488000    tuesday log account review go activity stateme...
19669      go today get ad consumer loan one main ad inst...
Name: Consumer_complaint, dtype: object

In [26]:
#dfNew.to_csv ('dfNew.csv', index = False, header=True)



Using TF-IDF-vectors, that have been calculated with the entire corpus (training and test subsets combined), while training the model might introduce some data leakage and hence yield in too optimistic performance measures. This is because the IDF-part of the training set's TF-IDF features will then include information from the test set already.

Calculating them completely separately for the training and test set is not a good idea either, because besides testing the quality of your model then you will be also testing the quality of your IDF-estimation. And because the test set is usually small this will be a poor estimation and will worsen your performance measures.

Therefore I would suggest (analogously to the common mean imputation of missing values) to perform TF-IDF-normalization on the training set seperately and then use the IDF-vector from the training set to calculate the TF-IDF vectors of the test set.


In [27]:
from sklearn.model_selection import train_test_split
y = dfNew['Product']
X=dfNew.Consumer_complaint
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

tfidf = TfidfVectorizer(max_features=2000, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
X_train_counts = tfidf.fit_transform(X_train)
X_test_counts = tfidf.transform(X_test)

TF-IDF Matrix for whole document

In [17]:

vectors = tfidf.fit_transform(dfNew.Consumer_complaint)
feature_names = tfidf.get_feature_names()
dense = vectors.todense()
denselist = dense.tolist()
df1 = pd.DataFrame(denselist, columns=feature_names)
df1.head(20)

,abide,ability,able,absolutely,absurd,abuse,abusive,ac,acc,accept,...,www,ye,year,yell,yesterday,yet,york,young,yr,zero
0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
1,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
2,0.0,0.0,0.125627,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
3,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
4,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
5,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
6,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
7,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
8,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
9,0.0,0.0,0.069831,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.107046,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
